# Trading Text-to-JSON

Python script that converts trading strategy text prompt to json.

## Loading dependencies

In [1]:
!pip install ollama

## Defining LLM call

We choose a large language model (LLM) and define how we will call the LLM. What we will input and output.

In [35]:
import ollama
import json
import re

model = 'gemma'
#model = 'mistral'
#model = 'llama3.3'
#model = 'tinyllama'

def ask(strategy, fields):
    prompt = """
Fill up the json below by the following prompt "{strategy}". Leave the json values by default if the json fields do not relate to the prompt. Do not add any new json fields, only use those json fields that are specified in my prompt.
{fields}
""".format(strategy=strategy, fields=fields)
    res = ollama.generate(model, prompt)
    #res = ollama.generate(model, prompt, format="json")
    print(res.response)

def ask_prompt(prompt):
    res = ollama.generate(model, prompt)
    print(res.response)

def ask_json(strategy, fields):
    prompt = """
Fill up the json below by the following prompt "{strategy}". Leave the json values by default if the json fields do not relate to the prompt. Do not add any new json fields, only use those json fields that are specified in my prompt.
{fields}. Output only JSON object. Make sure the JSON object output is wrapped in curly brackets.
""".format(strategy=strategy, fields=fields)
    
    #res = ollama.generate(model, prompt)
    #res = ollama.generate(model, prompt, format="json")
    #str = res.response
    
    res = ollama.chat(model, messages=[{'role': 'user', 'content': prompt}], format='json')
    json_obj = json.loads(res.message.content)
    return json_obj

## Defining Test Trading Strategies

We define the list of the test trading strategies. For now, we choose only one strategy from the list to test the script. We could test different strategies in a loop.

In [36]:
strategies = [
  "Sell 3% of portfolio but not more than 30 coins, keep 3%, tolerate 1%.",
  "Sell 25 coins, aiming to reduce portfolio by 3%, tolerating a deviation of 1%.",
  "Reduce portfolio by **3%** by selling a maximum of **20** coins, allowing for a 1% deviation from the target.",
  "Sell **5-15** coins to achieve a 3% portfolio reduction, tolerating a 1% deviation.",
  "Limit selling to **25-30** coins while reducing portfolio by 3%, allowing for a 1% margin of error.",
  "Determine the number of coins to sell to achieve a 3% reduction while staying within a 1% tolerance level.",
  "Sell enough coins to reduce portfolio by **3%** without selling more than **28** assets.",
  "Reduce portfolio by **3%** by selling a number of coins between **10-20**, accepting a 1% deviation.",
  "Determine the optimal number of coins to sell to achieve a 3% reduction while staying within a 1% tolerance.",
  "Sell **15-20** assets to achieve a 3% reduction in portfolio value, allowing for a 1% margin of error.",
  "Reduce portfolio by **3%** by selling a maximum of **30** assets, accepting a 1% deviation from the target.",
]

strategy = strategies[1]
print('strategy:', strategy)

strategy: Sell 25 coins, aiming to reduce portfolio by 3%, tolerating a deviation of 1%.


## Defining JSON fields

We define JSONs for trading on backend. The JSON fields are described with comments that explain each value. We use small chunks of the resulting JSON so that the LLM will be able to process each chunk simpler. 

The original JSON is described in the following project:               
[https://github.com/trademire/strategy-rapidhunter-en](https://github.com/trademire/strategy-rapidhunter-en)

In [37]:
portfolio_asset = """{
    "portfolioAsset": "USDT",   // Describes which asset to be used to calculate the overall portfolio (i.e., "USDT"), and by default, competition will take place in its related available markets (i.e., "X/USDT", "Y/USDT")
    "useBuySide": true,         // If enabled (`true`) competition in buy side will take place in related assets, otherwise (`false`) no competition will take place in any of assets and existing buy orders are cancelled.
    "useSellSide": true,        // If enabled (`true`) competition in sell side will take place in related assets, otherwise (`false`) no competition will take place in any of assets and existing sell orders are cancelled.
    "fakeMode": true,           // If enabled (`true`) it does not submit any  create nor cancel any order request to the actual exchange, instead it performs all neccessary computations and gathers data in a sort of emulated mode. If fake mode is enabled, the strategy stops automatically (and intentionally) after a fixed number of rounds.
    "insistBuySideIfHold": false,  // If enabled (`true`) and an amount of an asset is held then the buy logic defined buy user is always enforced for that asset, otherwise  (`false`) the strategy checks buy logic of assets that are only feasible (passing the filters defined by users). The standard behaviour is to keep this flag disabled and let the strategy to dynamically decide which market to enter and perform buy logic, however in some edge cases, users might be willing to force  (and execute buy logic) in some scenarios. 
    "radarHours": 0  // Rapid Hunter keeps track of temporal metrics (defined by the user, see next sections) in a moving window fashion, measuring remote data in a time-series manner, which are then used to construct complex indicators dictated by the users. While these measurements take place automatically, and for all assets, in order to gain speed and reduce the complexity, some of these assets after certain time stop being measured automatically (i.e., if no buy opportunity, etc) . Radar hours parameter allows users to tell strategy how many hours to still keep track of metrics of these outdated assets. Such a parameter can be useful especially when an asset that is not interesting might have a change to become relavent again in a given time period, thus having these measurements already in place. 
}"""

forbidden_assets = """{
"forbidden": {
  "newListingDays": 1.2, // Assets listed within 1.2 days are considered newlisted, therefore no competition would take place
  "assetsSpecific": {    // Specifies which assets (thus their related markets) are forbidden to compete
    "BTC": "buy",        // Forbids competition in buy side for BTC (i.e., BTC/USDT, BTC/ETH, BTC/DOGE, etc) 
    "ETH_B": "both",     // Forbids competition in both sides (buy and sell) for ETH_B asset
    "XRP_S": "both"      // ..
  },
  "assetsPostfix": {    // Specifices which assets matching the postfix are forbidden to compete
    "3L": "sell",       // Assets those names ending with "3L" are forbidden to compete in sell side
    "3S": "both",       // Assets those names ending with "3S" are forbidden to compete in both sides
    "4L": "both",       // ...
    "4S": "both"        // ...
  }
},
"""

fixed_assets = """{
"fixed": {
  "enabled": false,       // Enables (`true`) or disables (`true`) competition in fixed assets
  "duplex": true,         // If enabled (`true`) an asset can be competed both in buy and sell side at the same side, otherwise (`false`) only single side at a time
  "onlyPartition": false, // Enable competition in fixed assets, only if these assets are in the selected partitions
  "onlyDynamic": false,   // Enable competition in fixed assets, only if these assets are also passing dynamic selection filters of price, volume, and spread
  "assetsSpecific": [     // It includes array of fixed assets to be competed 
    "BTC"                 // Competition to buy and/or sell of BTC is allowed in BTC/USDT market (as USDT is the portfolio asset), if market exists
  ],
  "assetsPrefixInterval": [ // For available list of assets in a target exchange, it allows alphabetic filter to be applied to select fixed markets
    "A",                  // Start alphabetically from assets those their first letter being A (i.e., A/USDT, B/USDT, ...)
    "C"                   // End alphabetically by assets those their first letter being C (i.e., ,... B/USDT, C/USDT)
  ],
  "tags": {               // It allows users to assign tags to a specific set of assets
    "ozel": [             // Define a tag arbitrarily called "ozel"
      "KCS"               // Asset with symbol "KCS" is tagged as "ozel"
    ]
  }  
},
"""

dynamic_assets = """{
    "dynamic": {
      "enabled": false,     // Enables (`true`) or disables (`true`) competition in dynamically filtered assets
      "duplex": true,      // If enabled (`true`) a dynamic asset can be competed both in buy and sell side at the same round, otherwise (`false`) only single side at a time
      "priceMin": 1e-8,    // Minimum price criterion should be: 1e-8 (meaning 0.00000001), AND
      "priceMax": "+",     // Maximum price criterion should be with no limites, AND
      "volumeMin": 0,      // Minimum volume criterion should 0.0
      "volumeMax": "+",    // Maximum volume criterion should be unlimited, AND
      "spreadMin": 1e-8,   // Minimum simple spread should be 0.00000001%, AND
      "spreadMax": "+"     // Maximum simple spread should be unlimited.
    },
},
"""

partition_price = """{
    "partitionByPrice": {
      "enabled": false,  // Enable assets partitioning by price (`true`) otherwise  (`false`) closed
      "pieces": 5,       // Divide all assets into 5 partitions (in ascending order of price) with equal number of members
      "select": [        // In these partitions, select assets of specified pieces/partitions.
        3,               // 3. piece/partition
        2                // ...
      ]
    },
},
"""

partition_volume = """{
    "partitionByVolume": {
      "enabled": false,    // Enable assets partitioning by volume (`true`) otherwise  (`false`) closed
      "pieces": 5,         // Having partitioned by price earlier (otherwise all assets in a single piece), further divide all given assets into 5 partitions (in ascending order of volume) with equal number of members
      "select": [           // In these partitions, select assets of specified pieces/partitions. The piece 0 is a special/extra piece representing assets with 0 or unknown volume.
        0,                  // Select assets with no volume
        2,                  // 2. piece/partition selected
        3,                  // ...
      ]
    },
},
"""

partition_spread = """{
    "partitionBySpread": {
      "enabled": false,    // Enable assets partitioning by simple spread (`true`) otherwise  (`false`) closed
      "pieces": 2,         // Having partitioned by price and volume earlier (otherwise all assets in a single piece), further divide all given assets into 5 partitions (in ascending order of simple spread) with equal number of members.
      "select": [          // In these partitions, select assets of specified pieces/partitions. The piece 0 is a special/extra piece representing assets with no simple spread.
        1,                 // 1. piece/partition selected
        2                  // ...
      ]
    },
},
"""

budget = """{
    "budget": {
      "portfolio":  "100%",       // Defines how much value in terms of portfolio unit (such as "USDT") can be consumed by the strategy while competing, either in terms of absolute value  (such as `100` USDT) or percentage (such as `"100%" USDT`)
      "dust": 1,                  // Defines value of dust amount as 1 USDT, thus any asset with value below this threshold is considered non existent. 
      "default": {                // Default consumable budget per asset (provided that there is enough budget in the vault, and no further restriction specified later on)   
          "compete": "5%",        // Consume up to 5% of portfolio while computing in buy and sell
          "max": 200,             // If compete amount is given as percentage, this parameter sets an upper bound in terms of absolute portfolio value in USDT. For no upper bound, use `"+"`.
          "tolerance": "1%",      // Compete budget in an asset can be overridden in the states logic depending on the situation, up to a tolerance value defined here, either in percentage or absolute value of USDT.
          "keep": 0               // Defines how much of a given asset can be kept without competition nor ordering, either in absolute value or percentage form (such as "10%").
      },
      "byVolumePartitions": {     // Allocates budget by volume partitions
        "4": {                    // Piece/partition number 4 specifications
          "compete": 50,          // Allocate up to 50 USDT for buy and sell in total
          "max": "+",             // No maximum limit
          "tolerance": 0.0,       // No tolerance for exceeding the budget
          "keep":  10.0           // Keep 10 USDT of the asset on the side, without any competition
        },
        "2": {                    // ..
          "compete": 5,           // ..
          "max": "+", 
          "tolerance": 0.0,
          "keep":  0.0 
        },
        "3": {
          "compete": 10,          // ..
          "max": "+", 
          "tolerance": 0.0,
          "keep":  0.0 
        }
      },
       "byTags": {                 // Allocates budget by specific tagged assets (overrides settings of budget by volume partitions, if overlaps)  
        "ozel": {                  // Define allocation for "ozel" tag
          "compete": 100,          
          "max": "+",             
          "tolerance": 0.0,       
          "keep":  500.0          
        }
      }, 
      "byAssetsSpecific": {       // Allocates budget by specific assets (overrides settings of budget by volume partitions and tags, if overlaps)
        "XRP": {                  // Define allocation for "XRP" asset
          "compete": 100,          
          "max": "+",             
          "tolerance": 0.0,       
          "keep":  500.0          
        }
      }
    },
}
"""

constant_metrics = """{
    // Constant metrics allow users to define a set of parameters which can be later used in other parts of profile, to avoid going over hard-coded values written in multiple places, thus reducing the errors and increasing the readability of the profile.
    "metricsConstant": {
      "hourMax": 100,                     // Arbitrarily define a metric called "hourMax" and set its value to 100.0
      "date1": "2022-09-27T18:00:00.000Z" // Specify a date in ISO 8601 format (https://www.iso.org/iso-8601-date-and-time-format.html). This date is automatically converted to Unix timestamp in seconds, in order to be able to use it in metrics manipulations (see https://www.epochconverter.com for an example of conversion)
    },
},
"""

simple_metrics = """{
    // Simple metrics enable users to gather public and private data from exchange and other sources, and are listed here (and in the profile) for documentation purposes. Their parameters can be altered but new metrics cannot be introduced by the users. Basic metrics are very fundemental to build more complex metrics named as derived, functional, temporal, alternative, and final metrics (all described in next sections).
    "metricsSimple": {
      "marketSummary": {                // Metric group about market summary and ticker. If target asset is "EJD", then the market summary is about "EJD/USDT" pair as our portfolio unit is USDT.
        "volume": true,                 // Volume in 24 hours
        "lastPrice": {                  // Last price
          "approximate": true           // If last price not available, enable approximation via "bidPrice" ve "askPrice"
        },
        "bestBid": true,                // Best bid price in buy side
        "bestAsk": true,                // Best ask price in sell side
        "bestPriceSideCurrent": true,   // Best price in current side (best bid price if competing in buy side, best ask price if competing sell side)
        "bestPriceSideCross": true,     // Best price in cross side (best ask price if competing in buy side, best bid price if competing sell side)
        "spreadSimple": true,           // Simple spread in terms of percentage: (100 * bestBid/bestAsk) - 100
        "lowPrice24H": true,            // Lowest price within 24 hours (not supported by all exchanges)
        "highPrice24H": true,           // Highest price within 24 hours (not supported by all exchanges)
        "change": true,                 // Change within 24 hours (last price now compared to open price)
        "partitionPrice": true,         // Calculated partition/piece number by price
        "partitionVolume": true,        // Calculated partition/piece number by volume 
        "partitionSpread": true         // Calculated partition/piece number by simple spread
      },
      "market": {               // Metric group about market limitations, i.e., for EJD/USDT pair if the current scanned asset is EJD.
        "minPrice": true,       // Minimum allowed pirce, i.e., 0.00000001 USDT
        "minQuantity": true,    // Minimum allowed quantity, i.e., 0.1 EJD
        "minTotal": true,       // Minimum allowed total (price * quantity), i.e., 0.2 USDT
        "tickPrice": true,      // Tick size for price increment, i.e., 0.00000001 USDT
        "tickQuantity": true,   // Tick size for quantity increment, i.e., 0.1 EJD
        "tickTotal": true,      // Tick size for total increment, i.e., 0.45 USDT
        "digitsPrice": true,    // Allowed price digits after decimal, i.e., 8 digits
        "digitsQuantity": true, // Allowed quantity digits after decimal, i.e., 1 digits
        "digitsTotal": true,    // Allowed total (price * quantity) digits after decimal, i.e., 2 digits
        "maxPrice": true,       // Maximum allowed price, i.e., 100.00000000 USDT
        "maxQuantity": true,    // Maximum allowed quantity, i.e., 5000.00 EJD
        "maxTotal": true        // Maximum allowed total (price * quantity), i.e., 1000000.0 USDT
      },
      "vaultPortfolio": {               // Metric group about total vault portfolio
        "available": true,              // Available portfolio in USDT, not including  other assets nor buy orders
        "held": true,                   // Total portfolio value held by buy orders
        "others": true,                 // Total portfolio value held by other assets than USDT
        "total": true                   // Total portfolio value
      },
      "vaultOrders": {                  // Metric group about total vault orders
        "nrOfBoths": true,              // Number of both buy and sell orders
        "nrOfBuys": true,               // Number of buy orders
        "nrOfSells": true               // Number of sell orders
      },
      "vaultBudget": {                  // Metric group about vault budget
        "current": true,                // Current budget (in terms of USDT) allocated to competition and orders
        "target": true                  // Target budget (in terms of USDT) desired to be allocated to competition and orders
      },
      "balance": {                      // Metric group about asset balances
        "available": true,              // Available asset balance (in terms of asset quantity)
        "held": true,                   // Balance held by sell orders  (in terms of asset quantity) and other reasons
        "total": true,                  // Total balance (in terms of asset quantity)
        "converted_available": true,    // Available asset balance (converted to portfolio unit USDT)
        "converted_held": true,         // Balance held by sell orders  (converted to portfolio unit USDT) and other reasons
        "converted_total": true,        // Available asset balance (converted to portfolio unit USDT)
        "avgBuyPrice": true             // Average buy price of the asset given trade history. If target coin is EJD, then EJD/USDT trade history is traversed to compute this value, in terms of portfolio unit.
      },  
      "budget": {                       // Metric group about budget assigned to target asset
        "buyCurrent": true,             // Current budget used by buy orders
        "buyTarget": true,              // Target budget for buy orders
        "sellCurrent": true,            // Current budget used by sell orders
        "sellTarget": true,             // Target budget for sell orders
        "keepCurrent": true,            // Current keep budget (value not used by sell orders)
        "keepTarget": true,             // Target keep budget (value not used by sell orders)
        "sumCurrent": true,             // Current sum budget (combination of current buy and sell budget)
        "sumTarget": true,              // Target sum budget (combination of target buy and sell budget)
        "dust": true                    // Calculated dust value (defined earlier by the user or dictated by the market limitations, whichever is higher)
      },
      "ordersSideCurrent": {            // Metrics group about target asset orders in current side (buy orders when in buy side, sell orders when in sell side)
        "best_price": true,             // Price among all orders of price group that is ranked as the best
        "best_quantity": true,          // Quantity among all orders of price group that is ranked as the best
        "best_total": true,             // Total (price * quantity) among all orders of price group that is ranked as the best
        "best_rank": true,              // Rank/ordering of price group that is ranked as best
        "avg_price": {                  // Average price of orders in a given rank interval
          "rankStart": 1,               // Rank start (use negative value to start from the end, i.e., -2)
          "rankEnd": "+"                // Rank end (use "+" to iterate until the end without any specific rank)
        }, 
        "sum_quantity": {               // Sum quantity of orders in a given rank interval
          "rankStart": 1,               // Rank start (use negative value to start from the end, i.e., -2)
          "rankEnd": "+"                // Rank end (use "+" to iterate until the end without any specific rank)
        },
        "sum_total": {                  // Sum total (price * quantity) of orders in a given rank interval
          "rankStart": 1,               // Rank start (use negative value to start from the end, i.e., -2)
          "rankEnd": "+"                // Rank end (use "+" to iterate until the end without any 
        },
        "worst_price": true,            // Price among all orders of price group that is ranked as the worst
        "worst_quantity": true,         // Quantity among all orders of price group that is ranked the worst
        "worst_total": true,            // Total (price * quantity) among all orders of price group that is ranked as the worst
        "worst_rank": true,             // Rank/ordering of price group that is ranked as the worst
        "nrOfOrders": true,             // Total number of orders independent than whether there are multiple orders in a given price or not
        "nrOfGroups": true,             // Total number of price groups occupied by orders
        "newest_durationHours": true,    // Duration since the newest created order, in fraction of hours
        "oldest_durationHours": true,    // Duration since the oldest created order, in fraction of hours
      },
      "ordersSideCross": {              // Metrics group about target asset orders in cross side (sell orders when in buy side, buy orders when in sell side)
        "best_price": true,             // Price among all orders of price group that is ranked as the best
        "best_quantity": true,          // Quantity among all orders of price group that is ranked as the best
        "best_total": true,             // Total (price * quantity) among all orders of price group that is ranked as the best
        "best_rank": true,              // Rank/ordering of price group that is ranked as best
        "avg_price": {                  // Average price of orders in a given rank interval
          "rankStart": 1,               // Rank start (use negative value to start from the end, i.e., -2)
          "rankEnd": "+"                // Rank end (use "+" to iterate until the end without any specific rank)
        }, 
        "sum_quantity": {               // Sum quantity of orders in a given rank interval
          "rankStart": 1,               // Rank start (use negative value to start from the end, i.e., -2)
          "rankEnd": "+"                // Rank end (use "+" to iterate until the end without any specific rank)
        },
        "sum_total": {                  // Sum total (price * quantity) of orders in a given rank interval
          "rankStart": 1,               // Rank start (use negative value to start from the end, i.e., -2)
          "rankEnd": "+"                // Rank end (use "+" to iterate until the end without any 
        },
        "worst_price": true,            // Price among all orders of price group that is ranked as the worst
        "worst_quantity": true,         // Quantity among all orders of price group that is ranked the worst
        "worst_total": true,            // Total (price * quantity) among all orders of price group that is ranked as the worst
        "worst_rank": true,             // Rank/ordering of price group that is ranked as the worst
        "nrOfOrders": true,             // Total number of orders independent than whether there are multiple orders in a given price or not
        "nrOfGroups": true,             // Total number of price groups occupied by orders
        "newest_durationHours": true,    // Duration since the newest created order, in fraction of hours
        "oldest_durationHours": true,    // Duration since the oldest created order, in fraction of hours
      },
      "ordersSideBuy": {
        // ...
        // Similar metrics to group of "ordersSideCurrent" but always computing for buy side
        // ...
      },
      "ordersSideSell": {
        // ...
        // Similar metrics to group of "ordersSideCurrent" but always computing for sell side
        // ...
      },
      "depthSideCurrent": {             // Metrics group about target asset public depth (orderbook) in current side (buy orders when in buy side, sell orders when in sell side)
        "avg_price": {                  // Average price in orderbook given ranking and other filters
          "rankStart": 1,               // Rank start (use negative value to start from the end, i.e., -2)
          "rankEnd": "+",               // Rank end (use "+" to iterate until the end without any specific rank)
          "count": 1,                   // Total number of price groups to consider starting from "rankStart"
          "my": true,                   // Consider my orders in target asset when computing
          "others": true                // Consider others' orders in target asset when computing
        },
        "sum_quantity": {               // Sum quantity in orderbook given ranking and other filters
          "rankStart": 1,               // Rank start (use negative value to start from the end, i.e., -2)
          "rankEnd": "+",                // Rank end (use "+" to iterate until the end without any specific rank)
          "count": 1,                   // Total number of price groups to consider starting from "rankStart"
          "my": true,                   // Consider my orders in target asset when computing
          "others": true                // Consider others' orders in target asset when computing
        },
        "sum_total": {                  // Sum total (price * quantity) in orderbook given ranking and other filters
          "rankStart": 1,               // Rank start (use negative value to start from the end, i.e., -2)
          "rankEnd": "+",                // Rank end (use "+" to iterate until the end without any specific rank)
          "count": 1,                   // Total number of price groups to consider starting from "rankStart"
          "my": true,                   // Consider my orders in target asset when computing
          "others": true                // Consider others' orders in target asset when computing
        },
        "estimatedPrice": {             // Estimated price in orderbook given a budget and filters
          "budget": "100%",             // Use all sum budget of target asset  (i.e., "100%") when emulating the consumption. Absolute value i.e., "1000" USDT can be also provided
          "othersOnly": true            // Only consider others' orders in consumption but not my orders (true), otherwise (false) 
        },
        "estimatedQuantity": {          // Estimated required quantity in orderbook given a price shake percentage and other filters
          "priceShakePercentage": 2.0,  // Target price shake percentage
          "othersOnly": true            // Only consider others' orders in consumption but not my orders (true), otherwise (false) 
        },
        "estimatedTotal": {             // Estimated required total (price * quantity) in orderbook given a price shake percentage and other filters
          "priceShakePercentage": 2.0,  // Target price shake percentage
          "othersOnly": true            // Only consider others' orders in consumption but not my orders (true), otherwise (false) 
        },
        "rankMax": true                 // Total number of ranks available in order book (note that while orderbook might be large, exchanges might return a limited number of depth due to resource and API limitations)
      },
      "depthSideCross": {               // Metrics group about target asset public depth (orderbook) in cross side (sell orders when in buy side, buy orders when in sell side)
        "avg_price": {                  // Average price in orderbook given ranking and other filters
          "rankStart": 1,               // Rank start (use negative value to start from the end, i.e., -2)
          "rankEnd": "+",                // Rank end (use "+" to iterate until the end without any specific rank)
          "count": 1,                   // Total number of price groups to consider starting from "rankStart"
          "my": true,                   // Consider my orders in target asset when computing
          "others": true                // Consider others' orders in target asset when computing
        },
        "sum_quantity": {               // Sum quantity in orderbook given ranking and other filters
          "rankStart": 1,               // Rank start (use negative value to start from the end, i.e., -2)
          "rankEnd": "+",                // Rank end (use "+" to iterate until the end without any specific rank)
          "count": 1,                   // Total number of price groups to consider starting from "rankStart"
          "my": true,                   // Consider my orders in target asset when computing
          "others": true                // Consider others' orders in target asset when computing
        },
        "sum_total": {                  // Sum total (price * quantity) in orderbook given ranking and other filters
          "rankStart": 1,               // Rank start (use negative value to start from the end, i.e., -2)
          "rankEnd": "+",                // Rank end (use "+" to iterate until the end without any specific rank)
          "count": 1,                   // Total number of price groups to consider starting from "rankStart"
          "my": true,                   // Consider my orders in target asset when computing
          "others": true                // Consider others' orders in target asset when computing
        },
        "estimatedPrice": {             // Estimated price in orderbook given a budget and filters
          "budget": "100%",             // Use all sum budget of target asset  (i.e., "100%") when emulating the consumption. Absolute value i.e., "1000" USDT can be also provided
          "othersOnly": true            // Only consider others' orders in consumption but not my orders (true), otherwise (false) 
        },
        "estimatedQuantity": {          // Estimated required quantity in orderbook given a price shake percentage and other filters
          "priceShakePercentage": 2.0,  // Target price shake percentage
          "othersOnly": true            // Only consider others' orders in consumption but not my orders (true), otherwise (false) 
        },
        "estimatedTotal": {             // Estimated required total (price * quantity) in orderbook given a price shake percentage and other filters
          "priceShakePercentage": 2.0,  // Target price shake percentage
          "othersOnly": true            // Only consider others' orders in consumption but not my orders (true), otherwise (false) 
        },
        "rankMax": true                 // Total number of ranks available in order book (note that while orderbook might be large, exchanges might return a limited number of depth due to resource and API limitations)
      },
      "depthSideBuy": {
        // ...
        // Similar metrics to group of "depthSideCurrent" but always computing for buy side
        // ...
      },
      "depthSideSell": {
        // ...
        // Similar metrics to group of "depthSideCurrent" but always computing for sell side
        // ...
      },
      "tradesSideCurrent": {          // Metrics group about target asset history of trades in current side (buy trades when in buy side, sell trades when in sell side)
        "last_price": true,           // Price of last trade
        "last_quantity": true,        // Quantity of last trade
        "last_total": true,           // Total (price * quantity) of last trade
        "last_durationHours": true,   // Duration since the last trade in terms of fractional hours
        "avg_priceByAge": {           // Average price of trades performed in a specific time window 
          "mins": 60                  // Calculates average price of trades happened in last 60 minutes
        },
        "avg_priceByBudget": {        // Average price of trades up to a specified budget iterated backward history 
          "percentage": 100           // Consider 100% of sum budget allocated to target asset and go backwards in history to calculate the average price
        },
        "durationHoursByBudget": {    // Duration since the oldest trade given budget constraint iterated backwards in history 
          "percentage": 100           // Consider 100% of sum budget allocated to target asset and go backwards in history to find the oldest trade then its duration
        }
      },
      "tradesSideCross": {            // Metrics group about target asset history of trades in cross side (sell trades when in buy side, buy trades when in sell side)
        "last_price": true,           // Price of last trade
        "last_quantity": true,        // Quantity of last trade
        "last_total": true,           // Total (price * quantity) of last trade
        "last_durationHours": true,   // Duration since the last trade in terms of fractional hours
        "avg_priceByAge": {           // Average price of trades performed in a specific time window 
          "mins": 60                  // Calculates average price of trades happened in last 60 minutes
        },
        "avg_priceByBudget": {        // Average price of trades up to a specified budget iterated backward history 
          "percentage": 100           // Consider 100% of sum budget allocated to target asset and go backwards in history to calculate the average price
        },
        "durationHoursByBudget": {    // Duration since the oldest trade given budget constraint iterated backwards in history 
          "percentage": 100           // Consider 100% of sum budget allocated to target asset and go backwards in history to find the oldest trade then its duration
        }
      },
      "tradesSideBuy": {
        // ...
        // Similar metrics to group of "tradesSideCurrent" but always computing for buy side
        // ...
      },
      "tradesSideSell": {
        // ...
        // Similar metrics to group of "tradesSideCurrent" but always computing for sell side
        // ...
      },
      "exchange": {                       // Metric group about general exchange statistics
        "volumeOfFeasibleAssets": true,   // Total volume of feasible assets (assets with target market i.e., EJD asset having EJD/USDT market)
        "volumeOfSelectedAssets": {       // Total volume of feasible assets that are selected by partitioning filter
          "fixed": true,                  // Include fixed filtered assets
          "dynamic": true,                // Include dynamically filtered assets
          "none": true                    // Include assets which are not fixed neither dynamic
        },
        "volumeOfUnselectedAssets": {     // Total volume of feasible assets that are not selected by partitioning filter
          "fixed": true,                  // Include fixed filtered assets
          "dynamic": true,                // Include dynamically filtered assets
          "none": true                    // Include assets which are not fixed neither dynamic
        },
        "volumeOfCompetingAssets": true,  // Total volume of feasible assets that are in competition (either having a quantity or open buy orders)
        "nrOfListedAssets": true,         // Number of listed assets with their target markets and not delisted
        "nrOfNoMarketAssets": true,       // Number of assets with no feasible target market
        "nrOfFeasibleAssets": true,       // Number of assets with feasible target market
        "nrOfSelectedAssets": {           // Number of feasible assets that are selected by partitioning filter
          "fixed": true,                  // Include fixed filtered assets
          "dynamic": true,                // Include dynamically filtered assets
          "none": true                    // Include assets which are not fixed neither dynamic
        },
        "nrOfUnselectedAssets": {         // Number of feasible assets that are not selected by partitioning filter
          "fixed": true,                  // Include fixed filtered assets
          "dynamic": true,                // Include dynamically filtered assets
          "none": true                    // Include assets which are not fixed neither dynamic
        },
        "nrOfCompetingAssets": true       // Number of feasible assets that are in competition (either having a quantity or open buy orders)
      },
      "now": {              // Metric group about current time (at the time of scanning the target asset)
        "totalSecs": true,  // Unix timestamp in seconds (i.e., 1734903655), meaning number of seconds since 1st of January 1970
        "seconds": true,    // Current seconds (0, 1, ..., 59)
        "minutes": true,    // Current minutes (0, 1, ..., 59)
        "hours": true,   	  // Current hours (0, 1, ..., 23)
        "day": true,        // Current day number (1, 2, .., 31)
        "dayOfWeek": true,  // Current day of week (1, 2, .., 7),  i.e., 1 is Monday, and 7 ise Sunday
        "week": true,			  // Current week of year (1, 2, ..., 52)
        "month": true,   	  // Current month number (1, 2, .., 12), i.e., 1 ise Ocak ve 12 ise Araliktir
        "year": true  		  // Current year (2022, 2023, ...
      },  
    },
},
"""

derived_metrics = """{
    // Derived metrics allow users to customize simple metrics by changing existing parameter values, and moreover with some additional parameters they can be used to measure values in other assets and markets. An example of structure is given as follows:
    "metricsDerived": {
      "depthSideCurrentAvgPrice5": {          // Arbitrarily define a derived metric name called as k "depthSideCurrentAvgPrice5"
        "use": "depthSideCurrent.avg_price",  // Use simple metric of "depthSideCurrent.avg_price" as target to customize
        "override": {                         // Set of parameters to customize is defined here
          "rankStart": 2,                     // Override rank start as 2
          "rankEnd": 3,                       // Override rank end as 3
          "count": "+",                       // Override count as unlimited
          "my": false,                        // Do not include my orders
          "others": true,                     // Include others orders
          "targetAsset": "ETH",   // [Optional] Target asset is automatic if not defined and takes the current asset symbol, otherwise forced to a specific symbol in a fixed manner 
          "targetMarket": "BTC"   // [Optional] Target market is automatic if not defined and takes the current portfolio asset symbol (i.e., USDT), otherwise forced to a specific symbol in a fixed manner. The symbol pair of targetAsset/targetMarket defines a target trading pair for the metric.
        }
      }
    },
    // As mentioned above, "override": {...} structure allows users to modify/override parameters of simple metrics, thus enabling them to have multiple metrics based on various configuration of simple metrics. Simple metrics without any parameter (i.e., bestBid in marketSummary group has only value of true) does not require any customization, except target asset and market (optional) described below.
},
"""

functional_metrics = """
// Functional metrics allow users to define sophisticated equations given built-in functions and metrics defined earlier (namely constant, simple, derived). An example structure is given as follows:
{
  "metricsFunctional": { 
    "potentialGainPercentage": "100.0 * (depthSideCurrentAvgPrice5/tradesSideCross.last_price) - 100.0",  // Arbitrarily name a functional metric as "potentialGainPercentage" and use previously defined metrics in the equation 
    "orderLossValue": "100.0 * (depthSideCurrent.best_othersPrice /tradesSideCross.last_price) - 100.0",  // ...
    "orderLossTarget": "5.0 + (10.0 - 0.1*max(tradesSideCross.last_durationHours, hourMax))",             // ...
    "rushWildLimit": "0.4 * tradesSideCross.last_durationHours"                                           // ...
  },
},
"""

temporal_metrics = """
// Temporal metrics allow users to measure values of simple, derived, and functional metrics in a moving time window, truly enabling users to design highly customized sophisticated indicators for decision making and analysis. An example of structure is given as follows:
{
    "metricsTemporal": {
      "lastPrice10Mins": {                  // Arbitrarily name our temporal metric group as "lastPrice10Mins"
        "use": "marketSummary.lastPrice",   // Use "marketSummary.lastPrice" metric for all related measurements
        "binSecs": 60,                      // Each bin in moving window should have size of 60 seconds
        "counts": 10,                       // Number of bins should be 10 (therefore 60 * 10 = 600 seconds or 10 minute of moving window)
        "minimumBins": 1,                   // Minimum number of required bins for computations is 1, otherwise returning NaN (not-a-number) value 
        "calculate": {                      // Calculations for "lastPrice10Mins" group is given in this structure
            "change": "change",                     // Arbitrarily name as "lastPrice10Mins.change",         and calculate change in whole window
            "mx": "high / 2.0",                     // Arbitrarily name this metric as "lastPrice10Mins.mx", and divide highest value of whole window by 2.
            "x1": "bin{1}.last + bin{2}.last",      // Arbitrarily name this metric as "lastPrice10Mins.x1", and sum up last prices of first and second oldest bins 
            "x2": "bins{1,10}.high",                // Arbitrarily name this metric as "lastPrice10Mins.x2", and find highest value of bins in range of 1-10 (included)
            "x3": "bins{1,2,10}.close",             // Arbitrarily name this metric as "lastPrice10Mins.x3", and find close value of bins in range of 1-10 (included) with step size of 2 
            "b2": "bins{open > close, 1}.avg",      // Arbitrarily name this metric as "lastPrice10Mins.b2", and find bins where opening value is higher than close value, then take average of last price of these selected bins
            "b3": "bins{open > close, 10}.avg"      // Arbitrarily name this metric as "lastPrice10Mins.b3", and find bins where opening value is higher than close value (provided that it can search most recent 10 bins), then take average of last price of these selected bins
        }
      }
    },
},
"""

alternative_metrics = """
// Alternative metrics allow users to assign alternative values/metrics to a target metric, in situations where the value of target metric is not available or undefined. This way, smooth operations of strategy in the absence of original value could be achieved, and whenever the value of target metric is available, it is then used.
{
    "metricsAlternative": {
      "lastPriceApprox": {                // Arbitrarily define an alternative metric named as "lastPriceApprox"
        "use": "marketSummary.lastPrice", // Use "marketSummary.lastPrice" as main metric
        "options": [                      // Options in the absence of value of main metric
          "marketSummary.bestBid",       // Alternatively "marketSummary.bestBid" can be used
          "marketSummary.bestAsk"        // Alternatively "marketSummary.bestAsk" can be used
        ],
        "type": "first",                  // Pick first available value of metrics from "options" according to ordering. For the moment, there is only "first" type.
        "freshnessSecs": "+"              // When alternative value is active (meaning no value for the main metric), how often that value should be refreshed in terms of seconds? For no refresh use "+".
      }
    },
},
"""

final_metrics = """
// Final metrikler vasitasiyla daha onceden tanimlanmis sabit, basit, turetilmis, fonksiyonel, temporal, ve alternatif metriklerin hepsi bir arada bir denklem icinde tanimlanip yeni bir metrik olusturulabilir. Ornek bir yapi su sekildedir:
{
  "metricsFinal": { 
    "indicator1": "min(lastPrice10Mins.mx, lastPriceApprox)",  // Arbitrarily define a final metric called "indicator1", and use two earlier defined metrics in its expression, resulting of minimum value of these two metrics
  },
},
"""

simple_limits = """
// Simple metrics are purely based on a reference metric plus an offset in terms of percentage.
{
    "limitsSimple": { 
      "limit1Bids": { // Arbitrarily define a simple limit as "limit1Bids"
        "refMetric": "depthSideCurrent.best_othersPrice", // Use "depthSideCurrent.best_othersPrice" metric as a reference point
        "percentageOffset": 4, // Add 4% more to value in sell side (or substract 4% more in buy side) to value returned by reference metric. The final value is then used as a limit point.
        "side": "buy"          // This limit can be used only in "buy" side. Other options are "sell" and "both"
      },
      "limit1Asks": {
        "refMetric": "depthSideCurrent.best_othersPrice",
        "percentageOffset": 2,
        "side": "sell"        // This limit can be used in "sell" side
      },
      "limitConditionalStay": {
        "refMetric": "tradesSideCross.last_price",
        "percentageOffset": 3,
        "side": "both"        // This limit can be used in "both" sides
      },
      "limitRushStay": {
        "refMetric": "tradesSideCross.last_price",
        "percentageOffset": 1,
        "side": "both"
      },
      "limitRushWild": { 
        "refMetric": "rushWildLimit",
        "percentageOffset": 0,
        "side": "both"
      }
    },
},
"""

functional_limits = """
// Functional limits allow users to define limits with complex set of equations that can include all kind of metrics.
{
    "limitsFunctional": {
      "limFun1": "100 * (balance.avgBuyPrice / marketSummary.bestBid) - 100.0", //  Arbitrarily define a functional limit as "limFun1"
    },
},
"""

custom_limits = """
// Custom limits enable users to define specific x/y points in a line, and remaining points are linearly interpolated. These x/y pairs is then used to determine the final limit value, by adding/subtracting y value to the price metric. This can be useful for scenarios where other types of limits are too complex or not sufficent for the desired behaviour in strategy.
{
    "limitsCustom": {
      "limitLuckyShot": { // Arbitrarily define a custom limit as "limitLuckyShot"
        "baseMetric": "tradesSideCross.durationHoursByBudget", // For x-axis, use "tradesSideCross.durationHoursByBudget" metric (base metric)
        "baseValues": [ // Specify expected points in x-axis 
          0,        // x = 0 (initial) point
          2,        // x = 2
          15,       // x = 15 
          60,       // x = ...
          360,      // x = ...
          3600,     // x = ...
          6000      // x = 6000 
        ],
        "priceMetric": "depthSideCurrent.best_othersPrice", // Use "depthSideCurrent.best_othersPrice" metric for (price metric)
        "percentageOffsets": [ // Defines y-axis in terms of percentage offset to be added/subtracted to value of price metric
         100,    // when x = 0 then y should be 100% worse than price metric value 
          50,    // when x = 2 then y should be 2% worse than price metric value
          5,     // when x = 15 then y should be 15% worse than price metric value
          1,     // when x = 60 ...
          -2,    // when x = 360 then y should be 2% better than price metric value
          -5,    // when x = 3600 ...
          -20    // when x = 6000 ...
        ],
        "side": "both" // This limit can be used in both sides
      }
    },
},
"""

simple_market_order_distributions = """
// Simple market order distributions are used to define single market orders, which can be used in both sides of competition.
{
    "ordersMarketSimple": {   // Main structure for simple market order distributions
      "oms01": {              // Define a distribution as "oms01" 
        "budget": 400.0,      // Use budget of 400 USDT, out of allocated budget to target asset (otherwise remaining budget)
        "side": "buy"         // Can be used in buy side
      },
      "oms02": {              // Define a distribution as "oms02" 
        "budget": "50%",      // Use budget of 50%, out of allocated budget to target asset (otherwise remaining budget)
        "side": "both"        // Can be used in both sides
      }
    },
}
"""

simple_limit_order_distributions = """
// Simple limit order distributions are used to define single limit orders, which can be used in both sides of competition.
{
    "ordersLimitSimple": {      // Main structure for simple limit order distributions
      "ols01": {                // Define a distribution as "ols01"
        "mode": "hard",         // Place orders in "hard" mode (other options are: "soft", "softer", "hard", "harder", "direct"). See below for details.
        "limits": {             // Specify  limits
          "inherit": true,      // Bu emir dagilmimini kullanacak hareketin sahip oldugu limitleri hesaba kat
          "innerExtra": [       // Specify more inner limits to this distribution
            // Empty               
          ],
          "outerExtra": [       // Specify more outer limits to this distribution
            // Empty
          ],
          "skippable": [        // Defines which limits are skippable for computation (when there is no value or not-a-number), otherwise strategy would pause
            // Empty
          ]
        },
        "price": "2.0%",        // Put a price 2% worse than the final/worst of inner limits  but not exceeding the final/worst of outer limits.
        "budget": 4.4,          // Use 4.4 USDT from the allocated budget given to target asset. Percentage values such as "10%" can be also used.
        "side": "both"          // Can be used in both sides of the competition
      }
    },
}
// In the structure above, ols01 distribution generates a price 2% worse than the final/worst of inner limits but not exceeding the final/worst of outer limits. For example in buy side, if final inner limit is 200 and final outer limit is 50, the price would be first computed as 200 - 200*0.02 = 196 USDT, then it will be approximated according to the mode specified. In fact, observe that order placement modes is set to hard mode which in fact a price approximation strategy, which takes into account the public orderbook. In particular, the following modes are possible:
// soft: Order price is computed locally according to its distribution, then using the public orderbook, the final price is approximated to nearest feasible price of others WITHOUT creating a new price group . If there is no existing price group found for approximation, a new price group is created.
// softer: Order price is computed locally according to its distribution, then using the public orderbook, the final price is approximated to nearest feasible price of others WITHOUT creating a new price group. If there is no existing price group found for approximation, the existing worst price group of others is used.
// hard: Order price is computed locally according to its distribution, then using the public orderbook, the final price is approximated to nearest feasible price of others but one tick better by creating a new price group. If there is no existing price group found for approximation, a new price group is created.
// harder: Order price is computed locally according to its distribution, then using the public orderbook, the final price is approximated to nearest feasible price of others but one tick better by creating a new price group. If there is no existing price group found for approximation, the existing worst price group of others but one tick better is used.
// direct: Order price is computed locally according to its distribution and it is its final price, without taking into account public orderbook.
// In summary, soft modes aim to follow existing prices of others, hard modes aim to beat the existing prices of others by having a one tick better offer, and direct mode has no consideration of others. Regarding to "soft vs softer", and "hard vs harder", observe that softer/harder versions take into others worst offers when nothing feasible to approximate in the orderbook, wherease "soft/hard" versions create a new price group directly in this situation. Note that due to API limitations, most exchanges do not provide a complete view of orderbook, thus such a differentation of "soft vs softer" and "hard vs harder" is used to further customize the order distributions in competition.
"""

functional_limit_order_distributions = """
// Functional limit order distributions are used to define single or multiple limit orders, according to price gap generation and budget distribution functions.
{
    "ordersLimitFunctional": {  // Main structure for functional limit order distributions
      "olf01": {                // Define a distribution as "ofs01"
        "mode": "soft",         // Use "soft" mode for the orders. See "Simple Limit Order Distributions" section for documentation of modes.
        "counts": 5,            // Generate up to 5 orders, if possible
        "side": "both",         // Can be used in both sides of competition (buy/sell)
        "budget": "100%",       // Allocate 100% of the budget to this distribution, from allocated budget of target asset
        "limits": {             // Specify  limits
          "inherit": true,      // Use existing limits of competition action (described in next sections) calling this distribution
          "innerExtra": [       // Specify more inner limits to this distribution
                                // ...               
          ],
          "outerExtra": [       // Specify more extra limits to this distribution
                                // ...
          ],
          "skippable": [        // Defines which limits are skippable for computation (when there is no value or not-a-number), otherwise strategy would pause
            // Empty
          ]
        },
        "variable": "x",              // Use variable "x" for generation functions of price and budget (i.e., x = 1, 2, 3, 4, 5)
        "price": {                    // Define price gap behaviour between orders
          "gapFunction": "1.0 * x",   // Starting from the final inner limit, place orders with the following gap function. In each iteration (x = 1 for first order, x = 2 for second order, ...) it worsens the price (decrease in buy side, increase in sell side)
          "unit": "%"                 // Gaps are specified in percentage compared to initial price (namely "%") otherwise (namely "val") absolute values in terms of portfolio asset USDT is considered. Absolute value should be avoided especially when competing in all assets of the exchange as USDT might lead to unexpected price offers. For fixed assets, however, it could be helpful.
        },
        "budgetDistribution": {       // Define budget distribution behaviour among orders
          "enabled": false,           // Not enabled (false) thus uniformly distributing budget across orders. Otherwise (false) customize the budget according to specifications here.
          "normalize":	true,         // If true, it normalizes summation of all values generated by "distFunction" below to 100% of the budget and compute the actual budget of each of them accordingly.
          "distFunction": "1.0 * x",  // Defines budget distribution function, with each iteration (x = 1 for first order, x = 2 for second order, ...) computing budget of the order 
          "unit": "%"                 // "%", "val", or "asset". It defines unit of value generated by distribution function in percentage (namely "%"), absolute values of portfolio asset (namely "val"), absolute value in terms of competing asset quantity (namely "asset") could be used. Note that "asset" should be carefully considered when competing in multiple assets, due to lack of generalization.
        }
      }
    },
}
"""

custom_limit_order_distributions = """
// Custom limit order distributions are used to define single or multiple limit orders, according to manually defined price gaps as well as manually defined budgets.
{
    "ordersLimitCustom": {  // Main structure for custom limit order distributions
      "olc01": {            // Define a distribution as "olc01"
        "mode": "soft",     // Use "soft" mode for the orders. See "Simple Limit Order Distributions" section for documentation of modes.
        "counts": 5,        // Generate up to 5 orders, if possible
        "side": "both",     // Can be used in both sides of competition (buy/sell)
        "budget": "100%",   // Allocate 100% of the budget to this distribution, from allocated budget of target asset
        "limits": {         // Specify  limits
          "inherit": true,  // Use existing limits of competition action (described in next sections) calling this distribution
          "innerExtra": [   // Specify more inner limits to this distribution
                            // ...
          ],
          "outerExtra": [   // Specify more outer limits to this distribution
                            // ...
          ],
          "skippable": [    // Defines which limits are skippable for computation (when there is no value or not-a-number), otherwise strategy would pause
            // Empty
          ]
        },
        "price": {          // Define price gap behaviour between orders
          "gaps": [         // Defines custom gap values in terms of percentage. The reference value starts from the final inner limit, and gaps are added/subtracted to determine the order prices. In cases of multiple orders but single gap definition, this gap is applied to all order prices.
            10.0            // Worsen 10% each time an order is computed
          ],
          "unit": "%"       // Gaps are specified in percentage compared to initial price (namely "%") otherwise (namely "val") absolute values in terms of portfolio asset USDT is considered. Absolute value should be avoided especially when competing in all assets of the exchange as USDT might lead to unexpected price offers. For fixed assets, however, it could be helpful.
        },
        "budgetDistribution": {  // Define budget distribution behaviour among orders
          "enabled": false,     // Not enabled (false) thus uniformly distributing budget across orders. Otherwise (false) customize the budget according to specifications here.
          "normalize":	true,   // If true, it normalizes all values specified in "vals" to 100% of the budget and compute the actual budget of them accordingly.
          "vals": [             // Defines budget values (to be normalized or not, in different units). If single value is specified but not 5, it is then assumed for all orders.
            2, 7, 3, 4, 6       // Assign 2, 7, 3, 4, 6 to budget distribution of five orders.
          ],
          "unit": "%"         // "%", "val", or "asset". It defines unit of value generated by distribution function in percentage (namely "%"), absolute values of portfolio asset (namely "val"), absolute value in terms of competing asset quantity (namely "asset") could be used. Note that "asset" should be carefully considered when competing in multiple assets, due to lack of generalization. 
        }
      }
    },
}
"""

simple_actions = """
// Simple actions can be used in the states logic (described in next sections), and can be performed on a given target set. New type of simple actions cannot be created by the users, but existing ones can be utilized for the logic.
{
    "actionsSimple": {          // Structure of simple actions
      "ordersSideCurrent": {    // Actions group about orders in current side (buy orders when in buy side, sell orders when in sell side)
        "cancel_all": true      // Cancel all existing current side orders of target asset
      },
      "ordersSideCross": {     // Actions group about orders in cross side (sell orders when in buy side, buy orders when in sell side)
        "cancel_all": true     // Cancel all existing cross side orders of target asset
      },
      "misc": {               // Various simple actions
        "skip": true,         // Skip processing current side of target asset
        "pause": true         // Pause strategy
      }
    },
}
"""

competition_actions = """
// Competition actions allow users to specify multiple order distributions on a specific target, given various constraints such as budget, speed/rate, inner and outer limits. Multiple competition actions can be defined by users, to be used in their competition logic in the states (described in next sections). The strategy aims to achieve a final order distribution on a target asset, dictated by the action definition and constraints, and submit create/cancel orders to exchange if needed, while optimizing the change complexity and overhead.
{
    "actionsCompetition": {       // Main structure for competition actions
      "competeWild": {            // Arbitrarily name this competition action as "competeWild"
        "budgetFactor":  1.2,     // Given a budget allocated to target asset, multiply it by factor of 1.2 to obtain the final budget (budget can be increased or reduced subject to tolerance values dictated earlier in main "budget" structure)
        "rateFactor":   2.0,      // Given a competition speed on a target asset, multiply existing rates with 2.0 to obtain the final rate limitations
        "limitsInner": [          // Use the following inner limits. The final inner limit from multiple limits is obtained by considering the worst value (minimum of all limits in buy side, maximum of all limits in sell side). If no limit is given, the best price in the related market is considered a limit, by default.
          "limit1Bids",
          "limit1Asks",
          "limitLuckyShot"
        ],
        "limitsOuter": [          // Use the following outer limits. The final outer limit from multiple limits is obtained by considering the worst value (maximum of all limits in buy side, minimum of all limits in sell side)
            // Empty
        ],
        "limitsSkippable": [      // Defines which limits are skippable for computation (when there is no value or not-a-number), otherwise strategy would pause
          // Empty
        ],
        "orderStepPercentage": 0.1,   // Unless there is more than 0.1% difference between computed or existing order in terms of price, no need of canceling existing one and create a new order. This parameter aims to minimize number of requests and order changes submitted to exchange, thus speeding up the competition and reducing overhead. 
        "orderCountPerPrice": 10,     // For a given price group, there can be up to 10 orders generated by order distributions (otherwise, merge neccessary orders to a bigger order at the same price group)
        "orderDistribution": [        // Use the following order distributions for this action. Note that this action, when executed, does not neccessarily create again all these distribution at the exchange side, but instead make the neccessary changes given the difference of computed target order grid and existing orders at the exchange.
          "ordersLimitFunctional",
          "ordersLimitCustom"
        ]
      }
    },
}
"""

states_and_start = """
// States concept is a very essential part of strategy design, allowing users to define their own finite state machines on target assets, thus giving possibility to have unlimited type of strategies in high speed, without any programming knowledge. In a given round during the strategy run, all assets of interest in the exchange are scanned and their states should be determined (first in sell side then in buy side) prior to taking an action. In a given round for a specific asset, the strategy starts from initial/begin state (described below) and check out the conditions, and depending on the result of the condition (true, false, unknown) next state can be checked or an action can be taken. This logic is totally up to user to design. The structure is given as follows:
{
    "states": {
      "wild": {                                       // Arbitrarily define a state named as "wild"
        "check": "potentialGainPercentage >= 10.0",   // Check whether the value of "potentialGainPercentage" for current asset is exceeding (or equal to) the value of 10.0
        "onTrue": "competeWild",                      // If check result is true, execute "competeWild" action
        "onFalse": "conditional",                     // If check result is false, go to "conditional" state
        "onUnknown": "misc.pause"                     // If check result is unknown, perform "misc.pause" action thus stopping the strategy
      },
      "conditional": {                                // Arbitrarily define a state named as "conditional"
        "check": "potentialGainPercentage >= 1.0",    // ...
        "onTrue": "misc.pause",                       // ...
        "onFalse": "rush",                            // ...
        "onUnknown": "misc.pause"                     // ...
      },
      "rush": {                                     // Arbitrarily define a state named as "rush"
        "check": "orderLossValue <= orderLossTarget && tradesSideCross.last_durationHours <= 100.0",
        "onTrue": "misc.pause",                     // ...
        "onFalse": "misc.pause",                    // ...
        "onUnknown": "misc.pause"                   // ...
      }
    },
}
"""

custom_scoring = """
// Default scoring of each asset is done by checking out the distance between actual budget (including amount held and orders) and target budget, therefore prioritizing some assets than others in a given round when distributing the available resources/budget. This behaviour can be customized via custom scoring structure defined below, in which users can define partitioning/grouping and scoring within group according to metrics defined earlier. This structure is in charge of these customizations.
{
    "scoringCustom": {                    // Structure for custom scoring
      "enabled": true,                    // Enable custom scoring (true), otherwise (false) disabled
      "partition": {                      // Partitioning strategy for dividing all assets into pieces/groups
          "enabled": true,                // Enable partitioning
          "sortBy": "budget.sumTarget",   // Use "budget.sumTarget" metric value of an asset for partitioning
          "sortIn": "desc",               // Use descending order (namely "desc"), otherwise ascending order (namely "asc") when using "budget.sumTarget" values for partitioning
          "pieces": 5,                    // Total of 5 pieces/partitions
          "pieceUnknown": 5,              // Place exclusively unknown values in the 5th partition (or it can be placed to the first piece, namely 1)
          "intervals": "auto"             // Based on range of "budget.sumTarget" values, divide automatically assets into 4 equal intervals (excluding one piece for unknowns). For custom specification of intervals, one can define four specific intervals i.e., [0, 10.0, 1000.0, 1000000.0] for five pieces 
      },
      "sortBy": "balance.converted_total",  // Within each piece/partition, sort assets by their "balance.converted_total" metric value
      "sortIn": "asc",                      // Use ascending order (namely "asc"), otherwise descending order (namely "desc") for sorting within the piece
      "sortUnknownLast": true,              // Assets with unknown sorting values should be placed to the end (true), otherwise beginning (false)
      "tags": {                             // In addition to custom partitioning and sorting for all assets, assets with specific tags could be exlusively handled with this structure
          "ozel": {                         // Define a scoring rule for assets tagged as "ozel"
              "enabled": true,                      // Enable this rule
              "piece": 1,                           // Place all tagged assets exlusively into 2 piece (1st piece was earlier used by unknown partitioning values thus cannot overlap with the piece number of tag) 
              "sortBy": "balance.converted_total",  // Within the piece number 2, use "balance.converted_total" metric value for sorting these assets
              "sortIn": "asc",                      // Use ascending order (namely "asc"), otherwise descending order (namely "desc") when using "balance.converted_total" values for sorting
              "sortUnknownLast": true               // Assets with unknown sorting values should be placed to the end (true), otherwise beginning (false)
          }
      }
    },
}
// As seen in the structure above, for the assets tagged as "ozel", we have placed them exlusively in piece/partition number 2, and observe that assets with no partition value are placed in number 5. Therefore, pieces of 2-4 are effectively used to perform custom scoring for the rest of assets in the exchange. When custom scoring is enabled, the strategy needs to initially measure the values of metrics used in the first round of competition (otherwise unexpected behaviour), therefore no action is expected to be performed. In the following rounds, the strategy perform its usual state check for each asset, and simultaneously updating the value of these metrics for custom scoring. Based on these updated values of these metrics, assets are ordered in the beginning of each round.
"""

rate_limitations = """
// Without any rate limitation, the strategy performs operations at the maximum speed allowed by the exchange request limitations, provided that local processing in the strategy is not a limiting factor. However, users might find it useful to customize the speed of competition per asset basis, especially for the strategy configurations where not-too-much changes is intentionally needed. When enabled, the strategy aims to achieve rate limitations given below per asset basis. In case of very fast rounds, these assets are scheduled to following rounds to meet the target speeds.
{
    "rateLimitations": {  // Main structure for rate limitations
        "sec": 0.01,    // Try to compete 0.01 times per second for target asset, AND
        "min": 6,       // Try to compete 6 times per minute for target asset, AND
        "hour": "+",    // Try to compete unlimited times per hour for target asset, AND
        "day": "+"      // Try to compete unlimited times per day for target asset.
      }
    },
}
"""

error_handling = """
// Errors caused by the strategy or requests to exchange would by default cause the strategy to pause its operations, which might be desirable or not. The structure below allows users to specify what kind of errors can be handled under which conditions, and depending on defined rules, it enables users to specify whether to pause or schedule/postpone competition in a future predefined time.
{
    "errorHandling": {              // Main structure for error handling
      "roundPauseThreshold": "20%", // If errors occur for 20% of competed assets within a given round, pause immediately. This is a global threshold to avoid situations where exchange is constantly having technical issues (i.e., maintenance, systematic errors, internal server errors, complete shutdown)
      "always": {                   // Structure for always handling any kind of error, independent than its error message
        "enabled": true,            // Enable this feature (true), otherwise (`false`) disabled
        "assets": "all",            // It should apply to all assets, otherwise specific assets in an array can be given i.e., ["EJD", "MEH", "SEL", "OZG"]
        "endpointOnly": true,       // Handle only endpoint/exchange errors (true), otherwise (false) it handles endpoint plus strategy related errors
        "skipAndScheduleSecs": 60,  // If an error is catched/handled, then skip and try target asset again in 60 seconds
      },
      "specific": {                   // Structure for handling specific errors based on their error message content
        "maintenance": {              // Name this handling arbitrarily as "maintenance"
          "enabled": true,            // Enable this handler
          "assets": "all",            // It should apply to all assets, otherwise specific assets in an array can be given i.e., ["EJD", "MEH", "SEL", "OZG"]
          "endpointOnly": true,       // Handle only endpoint/exchange errors (true), otherwise (false) it handles endpoint plus strategy related errors
          "catches": ["maintenance"], // If there is a case-sensitive text of "maintenance" in error message, then it can handle the error. For multiplhe phrases spread across in a message, use an array such as ["phrase1", "phrase2"]
          "skipAndScheduleSecs": 60  // If an error is catched/handled, then skip and try target asset again in 60 seconds
        }
      }
    }
}
"""

console_settings = """
// All operations happening during the round, as well as global view of competetition is reported to console (if enabled). The structure below allows users to customize the console settings in a way that a better debugging and analysis can be performed. Users can monitor evolution of competition for assets, and check out the related metrics, order distributions, actions, states, and so on.
{
    "console": {                // Main structure for console settings
      "level": "competing",     // Level of details for all assets, showing only details of assets that are currently competing. Other options are 1) "none" for showing nothing 2) "basic" just simple info of all assets, 3) "orderChange" for detailed info when order change happens, 4) "competing" when competition takes place independent than order change or not, 5) "detailed" for all details in a debbugging fashion. In general it is good to keep the level as minimum as possible, for sake of clarity in console messages.
      "levelByAssets": {        // Level of details for specific assets
        "XRP": "detailed"       // Show always operations of "XRP" asset in details
      },
      "measurementVisibility": "none", // Do not show ("none") any details of measurements used for calculation of temporal metrics, in terms of bins and windows. For showing simple information, use "basic". For very detailed information, use "detailed". This is helpful for debugging and tracking the measurements.   
      "measurementMetricGroup": "all",  // In case measurements are displayed, show "all" related metric groups. Otherwise, for specific display of measurements, one can use a array format such as ["lastPrice10Mins", "group2", "group3"]
      "columnsSimple": {        // Simple columns of console table shown during competition, which can be customized depending. These columns can be displayed (true) or hidden (false)
        "market": true,         // market 
        "type": true,           // Asset competition type (and tag name if any)
        "score": true,          // Score
        "buyStatus": true,      // Last buy state and action (if any)
        "sellStatus": true,     // Last sell state and action (if any)
        "buyBudget": false,     // Current and target buy budget
        "sellBudget": false,    // Current and target sell budget
        "keepBudget": false,    // Current and target keep budget
        "dustBudget": false,    // Dust budget
        "sumBudget": true,      // Sum budget
        "partition": false,     // Partitioning information (price, volume, spread)
        "listing": false,       // Number of days since its listing
        "duration": true,       // Duration of competition for target asset
        "requests": false,      // Number of exchange requests performed
        "rate": false,          // Rate of competition
        "age": false            // Age of competition (i.e., 2 seconds ago)
      },
      "columnsCustom": {                      // In addition to simple columns, users can also display custom columns based on metrics and limits, for debugging and analysis purposes
        "testKolonu": {                       // Arbitrarily create a custom column named as "testKolonu" 
          "enabled": true,                    // Enable (true) displaying this column, otherwise (false) hidden
          "use": "marketSummary.bestBid",     // Display value of "marketSummary.bestBid" metric
          "lazy": true,                       // Lazy mode, which does not explicitly perform computations (nor sending requests) but instead show the value if such a metric is computed earlier during the competition. If not lazy, neccessary computations/requests might be incurred
          "precision": 8,                     // Precision after decimal point
          "rounding": false,                  // Enable (true) or disable (false) rounding
          "beautify": false                   // Beautify (true) or keep as it is (false). It can be usedul to display large numbers
        }
      }
    }
}
"""

reporting = """
// Periodic reporting of metrics to the platform can be defined in the structure below. Note that for a metric to be reportable, it should be available/measured/calculated earlier according to strategy logic defined earlier, otherwise no reporting.
{
    "reporting": {
      "enabled": true,    // Enable (true) or disable (false) reporting of metrics
      "mode": "lazy",     // Reporting mode of "lazy" or "eager". Lazy mode metrics are reported if calculated earlier during state tree computations, whereas eager mode reporting explicitly calculates metrics in competing/selected assets or non-selected assets subject to radar hours.
      "onlySpecificAssets": true, // Report only values of specific assets with metrics declared below (true), otherwise (false) report all eligible assets given the operating mode + always force reporting of specific assets
      "metrics": {        // Map of all reportable metrics with "name": true/false format, where false means no reporting
        "marketSummary.lastPrice": false, // Reporting of simple metric "marketSummary.lastPrice" is defined but no reporting
        "rushWildLimit": true,  // Report "rushWildLimit" from functional metrics, whenever there is a measurement on a given asset
        "lastPrice10Mins.x1": true  // Report "lastPrice10Mins.x1" from temporal metrics, whenever there is a measurement on a given asset
      },
      "specificAssets": { // Map of assets that we explicitly like to calculate and report to the platform
        "BTC": true,    // Enable metric reporting of "BTC" asset for the metrics defined above
        "ETH": false    // Define metric reporting of "ETH" but not activated (false)
      }
    }
}
"""

## Requesting LLM for text-to-json

We prompt LLM with each chunk of the JSON. LLM can figure out the proper values from the trading strategy prompt.

In [38]:
# The following script can be optimized by running functions asyncroniously in parallel. 

portfolio_asset_obj = ask_json(strategy, portfolio_asset)
print("portfolio_asset_obj:", json.dumps(portfolio_asset_obj, indent=2))
print()

forbidden_assets_obj = ask_json(strategy, forbidden_assets)
print("forbidden_assets_obj:", json.dumps(forbidden_assets_obj, indent=2))
print()

fixed_assets_obj = ask_json(strategy, fixed_assets)
print("fixed_assets_obj:", json.dumps(fixed_assets_obj, indent=2))
print()

dynamic_assets_obj = ask_json(strategy, dynamic_assets)
print("dynamic_assets_obj:", json.dumps(dynamic_assets_obj, indent=2))
print()

partition_price_obj = ask_json(strategy, partition_price)
print("partition_price_obj:", json.dumps(partition_price_obj, indent=2))
print()

partition_volume_obj = ask_json(strategy, partition_volume)
print("partition_volume_obj:", json.dumps(partition_volume_obj, indent=2))
print()

partition_spread_obj = ask_json(strategy, partition_spread)
print("partition_spread_obj:", json.dumps(partition_spread_obj, indent=2))
print()

budget_obj = ask_json(strategy, budget)
print("budget_obj:", json.dumps(budget_obj, indent=2))
print()

constant_metrics_obj = ask_json(strategy, constant_metrics)
print("constant_metrics_obj:", json.dumps(constant_metrics_obj, indent=2))
print()

simple_metrics_obj = ask_json(strategy, simple_metrics)
print("simple_metrics_obj:", json.dumps(simple_metrics_obj, indent=2))
print()

derived_metrics_obj = ask_json(strategy, derived_metrics)
print("derived_metrics_obj:", json.dumps(derived_metrics_obj, indent=2))
print()

functional_metrics_obj = ask_json(strategy, functional_metrics)
print("functional_metrics_obj:", json.dumps(functional_metrics_obj, indent=2))
print()

temporal_metrics_obj = ask_json(strategy, temporal_metrics)
print("temporal_metrics_obj:", json.dumps(temporal_metrics_obj, indent=2))
print()

alternative_metrics_obj = ask_json(strategy, alternative_metrics)
print("alternative_metrics_obj:", json.dumps(alternative_metrics_obj, indent=2))
print()

final_metrics_obj = ask_json(strategy, final_metrics)
print("final_metrics_obj:", json.dumps(final_metrics_obj, indent=2))
print()

simple_limits_obj = ask_json(strategy, simple_limits)
print("simple_limits_obj:", json.dumps(simple_limits_obj, indent=2))
print()

functional_limits_obj = ask_json(strategy, functional_limits)
print("functional_limits_obj:", json.dumps(functional_limits_obj, indent=2))
print()

custom_limits_obj = ask_json(strategy, custom_limits)
print("custom_limits_obj:", json.dumps(custom_limits_obj, indent=2))
print()

simple_market_order_distributions_obj = ask_json(strategy, simple_market_order_distributions)
print("simple_market_order_distributions_obj:", json.dumps(simple_market_order_distributions_obj, indent=2))
print()

simple_limit_order_distributions_obj = ask_json(strategy, simple_limit_order_distributions)
print("simple_limit_order_distributions_obj:", json.dumps(simple_limit_order_distributions_obj, indent=2))
print()

functional_limit_order_distributions_obj = ask_json(strategy, functional_limit_order_distributions)
print("functional_limit_order_distributions_obj:", json.dumps(functional_limit_order_distributions_obj, indent=2))
print()

custom_limit_order_distributions_obj = ask_json(strategy, custom_limit_order_distributions)
print("custom_limit_order_distributions_obj:", json.dumps(custom_limit_order_distributions_obj, indent=2))
print()

simple_actions_obj = ask_json(strategy, simple_actions)
print("simple_actions_obj:", json.dumps(simple_actions_obj, indent=2))
print()

competition_actions_obj = ask_json(strategy, competition_actions)
print("competition_actions_obj:", json.dumps(competition_actions_obj, indent=2))
print()

states_and_start_obj = ask_json(strategy, states_and_start)
print("states_and_start_obj:", json.dumps(states_and_start_obj, indent=2))
print()

custom_scoring_obj = ask_json(strategy, custom_scoring)
print("custom_scoring_obj:", json.dumps(custom_scoring_obj, indent=2))
print()

rate_limitations_obj = ask_json(strategy, rate_limitations)
print("rate_limitations_obj:", json.dumps(rate_limitations_obj, indent=2))
print()

error_handling_obj = ask_json(strategy, error_handling)
print("error_handling_obj:", json.dumps(error_handling_obj, indent=2))
print()

console_settings_obj = ask_json(strategy, console_settings)
print("console_settings_obj:", json.dumps(console_settings_obj, indent=2))
print()

reporting_obj = ask_json(strategy, reporting)
print("reporting_obj:", json.dumps(reporting_obj, indent=2))
print()

print('DONE')

portfolio_asset_obj: {
  "portfolioAsset": "USDT",
  "useBuySide": true,
  "useSellSide": true,
  "fakeMode": true,
  "insistBuySideIfHold": false,
  "radarHours": 0,
  "sellAmount": 25,
  "sellTarget": 0.03,
  "sellDeviation": 0.01
}

forbidden_assets_obj: {
  "forbidden": {
    "newListingDays": 1.2,
    "assetsSpecific": {
      "BTC": "buy",
      "ETH_B": "both",
      "XRP_S": "both"
    },
    "assetsPostfix": {
      "3L": "sell",
      "3S": "both",
      "4L": "both",
      "4S": "both"
    }
  },
  "trade": {
    "action": "sell",
    "amount": 25,
    "target": "portfolio",
    "percent": -0.03,
    "tolerance": 0.01
  }
}

fixed_assets_obj: {
  "fixed": {
    "enabled": false,
    "duplex": true,
    "onlyPartition": false,
    "onlyDynamic": false,
    "assetsSpecific": [
      "BTC"
    ],
    "assetsPrefixInterval": [
      "A",
      "C"
    ],
    "tags": {
      "ozel": [
        "KCS"
      ]
    }
  }
}

dynamic_assets_obj: {
  "dynamic": {
    "enabled": false,
  

## Building the Final JSON

Now we merge small objects to a final JSON that can be send to backend for selling/buying the financial instruments.

In [39]:
final_json = {
    **portfolio_asset_obj,
    **forbidden_assets_obj,
    **fixed_assets_obj,
    **dynamic_assets_obj,
    **partition_price_obj,
    **partition_volume_obj,
    **partition_spread_obj,
    **budget_obj,
    **constant_metrics_obj,
    **simple_metrics_obj,
    **derived_metrics_obj,
    **functional_metrics_obj,
    **temporal_metrics_obj,
    **alternative_metrics_obj,
    **final_metrics_obj,
    **simple_limits_obj,
    **functional_limits_obj,
    **custom_limits_obj,
    **simple_market_order_distributions_obj,
    **simple_limit_order_distributions_obj,
    **functional_limit_order_distributions_obj,
    **custom_limit_order_distributions_obj,
    **simple_actions_obj,
    **competition_actions_obj,
    **states_and_start_obj,
    **custom_scoring_obj,
    **rate_limitations_obj,
    **error_handling_obj,
    **console_settings_obj,
    **reporting_obj,
}

print('final_json:', json.dumps(final_json, indent=2))

final_json: {
  "portfolioAsset": "USDT",
  "useBuySide": true,
  "useSellSide": true,
  "fakeMode": true,
  "insistBuySideIfHold": false,
  "radarHours": 0,
  "sellAmount": 25,
  "sellTarget": 0.03,
  "sellDeviation": 0.01,
  "forbidden": {
    "newListingDays": 1.2,
    "assetsSpecific": {
      "BTC": "buy",
      "ETH_B": "both",
      "XRP_S": "both"
    },
    "assetsPostfix": {
      "3L": "sell",
      "3S": "both",
      "4L": "both",
      "4S": "both"
    }
  },
  "trade": {
    "action": "sell",
    "quantity": 25,
    "target": "coins",
    "price": null,
    "commission": null,
    "deviation": 0.01,
    "impact": -0.03
  },
  "fixed": {
    "enabled": false,
    "duplex": true,
    "onlyPartition": false,
    "onlyDynamic": false,
    "assetsSpecific": [
      "BTC"
    ],
    "assetsPrefixInterval": [
      "A",
      "C"
    ],
    "tags": {
      "ozel": [
        "KCS"
      ]
    }
  },
  "dynamic": {
    "enabled": false,
    "duplex": true,
    "priceMin": 1e-08,
